# Import Library

In [23]:
# To run the codes, one will need to install the following libraries.

library(data.table)
library(dplyr)
library(lubridate)
library(cosinor2)
library(car)
library(olsrr)
library(writexl)
library(readxl)
library(ggplot2)
library(stringr)

# Extracting rhythms

In [27]:
date_time = "%Y-%m-%dT%H:%M:%OS"
period_in_hour <- 24
data_name = 'HeartRatePPG'

loc_save_file = 'C:/Users/wyd2hu/OneDrive - University of Virginia/Hridita/Rhythms/Findings/'
ds_loc_list <- c('C:/Users/wyd2hu/S2He/AudData/Tiles18/fitbit/heart-rate/') # 'C:/Users/wyd2hu/S2He/AudData/Tiles19/tiles-phase2-opendataset/fitbit/heart-rate/'

dataset = ''

for (loc_root_data in ds_loc_list)
{
    n_counter_less_than_7 = 0
    not_more_than_8_days = 0
    all_data_same = 0
    
    if (str_detect(loc_root_data, 'Tiles19'))
    {
        dataset = 'Tiles19'
    }
    else if (str_detect(loc_root_data, 'Tiles18'))
    {
        dataset = 'Tiles18'
    }

    print(paste0('DS ', dataset))
    saving_file <-  paste0(loc_save_file, dataset, '_', data_name,'.xlsx')
    rhythm_df_sheets <- list()

    # Checking whether rhythm is already extracted
    if (file.exists(saving_file))
    {
      saved_rhythm_sheets_by_data = excel_sheets(path=saving_file)
      for (rhythm_sheet in saved_rhythm_sheets_by_data)
      {
          rhythm_df_sheets[[rhythm_sheet]] <- read_excel(path = saving_file, sheet = rhythm_sheet)
      }
    }
    else
    {
        saved_rhythm_sheets_by_data = c()
    }
    
    for(file in list.files(loc_root_data, pattern = ".csv.gz"))
    {
        # if (grepl('16812063-e5df-4657-b86c-5b55a0c9ffe6', file, fixed=TRUE))
        # {
        print(paste0(dataset, ' ',  file))

        print(length(grep(str_sub(gsub('.csv.gz', '', file), 1, 25), saved_rhythm_sheets_by_data)))
        if (length(grep(str_sub(gsub('.csv.gz', '', file), 1, 25), saved_rhythm_sheets_by_data)) == 0)
        {
            df = fread(paste0(loc_root_data, "//", file))
            df$Timestamp <- as.POSIXct(df$Timestamp , format = "%Y-%m-%dT%H:%M:%OS")
            # df$Timestamp <- format(strptime(df$Timestamp, format = "%Y-%m-%d %H:%M:%OS"), format = "%Y-%m-%d %H:%M:%OS")
            df$time_float <- format(strptime(df$Timestamp, format = "%Y-%m-%d %H:%M:%OS"), format = "%H.%M")
            df$time_float <- as.numeric(df$time_float)
            
            df <- na.omit(df[, , drop = FALSE])
            if (nrow(df) > 1) # ensuring that there remains at least 1 value after dropping the nan values
            {
                first_row <- head(df, 1)
                last_row <- tail(df, 1)
                # print(last_row)
            
                start_date = first_row$Timestamp[[1]]
                
                d_days = as.integer(difftime(last_row$Timestamp[[1]] , start_date, units = "days"))
                n_times_rhythm_cal = as.integer(d_days/2)
        
                if (n_times_rhythm_cal > 4) # We need at least 7 days (2 days for rhythm. so, if we take 7 days, we will have 4 data points which I assume can be the minimum for estimating elbow) 
                    # to find the baseline rhythm to detect changes
                {
                    df_cosinor <- data.frame(matrix(ncol = 10, nrow = 1))
                    colnames(df_cosinor) <- c('N_Times', 'data_name', 'p_id', 'mesor', 'amplitude', 'correct_acrophase',
                                              'rhythm_detect_F_ratio', 'rhythm_detect_P', 'dominant_period', 'coef_determination')
                    # print(paste0("N times rhythm calculation ", n_times_rhythm_cal))
                    for(counter in 1:n_times_rhythm_cal)
                    {
                            # print(paste0(counter, start_date))
                            date_time <- as.POSIXct(start_date, format = "%Y-%m-%d %H:%M:%OS")
                            end_date = format(date_time + days(2), format = "%Y-%m-%d %H:%M:%OS")
                        
                            df_subset <- subset(df, subset=c((Timestamp >= start_date) & (Timestamp < end_date)))  ## the end date is going to be the start date in the next iteration
    
                            if (nrow(df_subset) > 7) # sample size must be greater than 7 to calculate the normality as per the req. of ols_test_normality
                            {
                                df_subset$time_in_24 = ifelse(df_subset$time_float < 1, df_subset$time_float+24, df_subset$time_float)
                                colnames(df_subset)[colnames(df_subset) == 'HeartRatePPG'] <- 'Data'

                                if (length(unique(df_subset$Data)) > 1)
                                {
                                        if (nrow(df_subset) > 5000) # since more than 5000 values, ols_test_normality() does not work
                                        {
                                            my_model <- lm(df_subset$Data[0:5000] ~ df_subset$time_in_24[0:5000])
                                        }
                                        else
                                        {
                                            my_model <- lm(df_subset$Data ~ df_subset$time_in_24)
                                        }
        
                                        kolmogorv_p <- ols_test_normality(my_model)[[1]][2] # [[1]][2] presents p value of Kolmogorov-Smirnov as I checked manually by this dataset and also by cars dataset
                                        if (kolmogorv_p < 0.05) # Rejected null hypothesis. So, data is not normally distributed :).
                                        {
                                            log_data = log(df_subset$Data + 1)
                                        }
                                        else # We do not have enough evidence to reject null hypothesis and data is normally distributed
                                        {
                                            log_data = df_subset$Data
                                        }
                                    
                                        fit <- cosinor.lm(log_data ~ time(time_in_24), data = df_subset, period = period_in_hour)
                                    
                                        s_fit <- summary(fit)
                                        mesor <- s_fit[[1]]$estimate[1] # Transformed coef: s_fit[[1]]; estimate column; s_fit[[1]]['estimate'][[1]] returns the values in the estimate column; [1] value is the intercept as I found through investigation
                                        ampli <- s_fit[[1]]$estimate[2]
                                        corr_acro <- correct.acrophase(fit)
                                        df_detect <- as.data.frame(cosinor.detect(fit))
                                    
                                        output <- periodogram(data = t(log_data), time = df_subset$time_in_24, periods=1:24)
                                        dominant_period <- which.max(unlist(c(output)$data))  # I checked it manually. MAJOR REMINDER: If you use the all periods (which is by default) given through (sheet_data$time_in_24), this will not work since the number of periods will be more 24 and indecies will not present the actual index
                            
                                        row_number = nrow(df_cosinor) + 1
                                        df_cosinor[row_number, ] <- c(counter, data_name, gsub('-', '', gsub(".csv.gz", '', file)), 
                                                                      mesor, ampli, corr_acro,
                                                                      df_detect$F, df_detect$p, dominant_period[[1]],
                                                                      unlist(c(output)$data)[dominant_period][[1]])
                                    
                                        # output <- output + scale_color_manual(values=c('Yellow')) + theme(
                                        #     axis.text.x = element_text(angle=90, colour="black", size=18, family='Times New Roman'),
                                        #     axis.text.y = element_text(colour="black", size=18, family='Times New Roman'),
                                        #     axis.title.x = element_text(colour= 'black', size=18, family='Times New Roman'),
                                        #     axis.title.y = element_text(colour = 'black', size=18, family='Times New Roman'),
                                        #     legend.title = element_blank(),
                                        #     panel.grid.major.x=element_line(colour="#CCDDEE"),
                                        #     legend.text = element_text(colour='black', family = 'Times New Roman', size=22),
                                        #     panel.grid = element_blank(), axis.ticks.y = element_line(),
                                        #     panel.background = element_rect(fill = 'white', colour = 'white'),
                                        #     plot.background = element_rect(fill = 'white', colour = 'white'))
                                        # # suppressMessages(ggsave(output, file=figure_to_saved, dpi=700))
                                        rhythm_df_sheets[[str_sub(gsub('.csv.gz', '', file), 1, 25)]] <- df_cosinor
                                }
                                else
                                {
                                    all_data_same = all_data_same + 1
                                }
                            }
                            else
                            {
                                n_counter_less_than_7 = n_counter_less_than_7 + 1
                            }
                            start_date = end_date
                    }
                }
                else
                {
                    not_more_than_8_days = not_more_than_8_days + 1
                    print("Less than or equal to 8 days data 😒😒")
                    print(file)
                }
            }
            else
            {
                print("Empty dataframe 😒😒")
                print(file)
            }
        }else
        {
            print(paste0('Exists', file))
        }
        # }
    }
    print(saving_file)
    write_xlsx(rhythm_df_sheets, saving_file) # REMINDER: if you use personalized dominant period, change period_in_hour
    print(n_counter_less_than_7)
    print(all_data_same)
    print(not_more_than_8_days)
    print('\n\n1 Dataset Done\n\n')
}

Your code contains a unicode char which cannot be displayed in your
current locale and R will silently convert it to an escaped form when the
R kernel executes this code. This can lead to subtle errors if you use
such chars to do comparisons. For more information, please see
https://github.com/IRkernel/repr/wiki/Problems-with-unicode-on-windows

[1] "DS Tiles18"
[1] "Tiles18 02581754-36cd-4b23-85ea-bf995c6dec83.csv.gz"
[1] 1
[1] "Exists02581754-36cd-4b23-85ea-bf995c6dec83.csv.gz"
[1] "Tiles18 0271c478-a56a-4c09-ab91-9743184dd71b.csv.gz"
[1] 1
[1] "Exists0271c478-a56a-4c09-ab91-9743184dd71b.csv.gz"
[1] "Tiles18 02b7a595-6508-46bd-8239-6deb433d6290.csv.gz"
[1] 1
[1] "Exists02b7a595-6508-46bd-8239-6deb433d6290.csv.gz"
[1] "Tiles18 05dedb61-63bc-44e3-8e28-a5d32d91f7e9.csv.gz"
[1] 1
[1] "Exists05dedb61-63bc-44e3-8e28-a5d32d91f7e9.csv.gz"
[1] "Tiles18 06b33ec4-706d-462f-a681-05491be38eb3.csv.gz"
[1] 1
[1] "Exists06b33ec4-706d-462f-a681-05491be38eb3.csv.gz"
[1] "Tiles18 0a85fd46-fada-434c-9f7a-08b81f9ed8e7.csv.gz"
[1] 1
[1] "Exists0a85fd46-fada-434c-9f7a-08b81f9ed8e7.csv.gz"
[1] "Tiles18 0adb7679-9d26-46e7-a134-11da293910f3.csv.gz"
[1] 1
[1] "Exists0adb7679-9d26-46e7-a134-11da293910f3.csv.gz"
[1] "Tiles18 0b45e9c1-eba5-46d3-b0a4-cdb5aa4dc736.csv.gz"
[1] 1
[1] "Exists0b45e9c1-eba5-46d3-b0a4-cdb5aa4dc736.csv.gz"
[1] "Tiles18 0bf54591-e

Warning message in `[.data.table`(df, , , drop = FALSE):
"i and j are both missing so ignoring the other arguments. This warning will be upgraded to error in future."

[1] "Less than or equal to 8 days data <U+0001F612><U+0001F612>"
[1] "62e88542-8d2e-4148-bd5f-2a3b47cdac44.csv.gz"
[1] "Tiles18 640d48a2-510a-49e6-bb3e-5bbd92222489.csv.gz"
[1] 1
[1] "Exists640d48a2-510a-49e6-bb3e-5bbd92222489.csv.gz"
[1] "Tiles18 658adbe4-781c-45f9-92a7-14912fcd0701.csv.gz"
[1] 1
[1] "Exists658adbe4-781c-45f9-92a7-14912fcd0701.csv.gz"
[1] "Tiles18 663de5e8-dfc8-42ad-8410-c196f6345bec.csv.gz"
[1] 1
[1] "Exists663de5e8-dfc8-42ad-8410-c196f6345bec.csv.gz"
[1] "Tiles18 665117c8-5dde-4655-a286-cd8ef2603d59.csv.gz"
[1] 1
[1] "Exists665117c8-5dde-4655-a286-cd8ef2603d59.csv.gz"
[1] "Tiles18 674e9815-7dd1-47f9-9916-c6e53d7e1a36.csv.gz"
[1] 1
[1] "Exists674e9815-7dd1-47f9-9916-c6e53d7e1a36.csv.gz"
[1] "Tiles18 67d47ef6-5f27-43cd-912a-a40f27a43660.csv.gz"
[1] 1
[1] "Exists67d47ef6-5f27-43cd-912a-a40f27a43660.csv.gz"
[1] "Tiles18 6812b3d0-5a2f-4d11-960a-23fc5c4e8c7b.csv.gz"
[1] 1
[1] "Exists6812b3d0-5a2f-4d11-960a-23fc5c4e8c7b.csv.gz"
[1] "Tiles18 687c2d53-012d-4082-a5f0-edade974

Warning message in `[.data.table`(df, , , drop = FALSE):
"i and j are both missing so ignoring the other arguments. This warning will be upgraded to error in future."

[1] "Less than or equal to 8 days data <U+0001F612><U+0001F612>"
[1] "803be457-cee4-4f3f-9540-ff574c57c697.csv.gz"
[1] "Tiles18 8307ff6e-f582-49b0-81cc-39fe9966d097.csv.gz"
[1] 1
[1] "Exists8307ff6e-f582-49b0-81cc-39fe9966d097.csv.gz"
[1] "Tiles18 8323bb7d-6b8d-4dea-82d6-08944e6da3a9.csv.gz"
[1] 1
[1] "Exists8323bb7d-6b8d-4dea-82d6-08944e6da3a9.csv.gz"
[1] "Tiles18 85c88df0-b503-4329-a29f-353199f7fe5b.csv.gz"
[1] 1
[1] "Exists85c88df0-b503-4329-a29f-353199f7fe5b.csv.gz"
[1] "Tiles18 883aca61-af06-4c14-ba58-8d2e4b75bfda.csv.gz"
[1] 1
[1] "Exists883aca61-af06-4c14-ba58-8d2e4b75bfda.csv.gz"
[1] "Tiles18 885170be-d945-427d-bd3a-b0524ddc3105.csv.gz"
[1] 1
[1] "Exists885170be-d945-427d-bd3a-b0524ddc3105.csv.gz"
[1] "Tiles18 89ff7373-ee85-4764-a4dd-90138af9af85.csv.gz"
[1] 1
[1] "Exists89ff7373-ee85-4764-a4dd-90138af9af85.csv.gz"
[1] "Tiles18 8a01ec4c-6222-4efd-9beb-0adb12e00a2b.csv.gz"
[1] 1
[1] "Exists8a01ec4c-6222-4efd-9beb-0adb12e00a2b.csv.gz"
[1] "Tiles18 8a2a2baf-826f-445d-a7fe-3a9b7f5d

In [20]:
test_list = list(1, 1, 1, 1, 1)
print(length(unique(test_list)))

[1] 1


In [ ]:
loc_save_file = 'C:/Users/wyd2hu/S2He/'
saving_file <-  paste0(loc_save_file, dataset, '_', data_name,'.xlsx')
print(saving_file)
write_xlsx(new_rhythm_df_sheets, saving_file) # REMINDER: if you use personalized dominant period, change period_in_hour

[1] "C:/Users/wyd2hu/S2He/Tiles18_HeartRatePPG.xlsx"


In [1]:
# new_rhythm_df_sheets = list()
for (p_id in names(new_rhythm_df_sheets))
{
    print(str_sub(p_id, 1, 20))
    new_rhythm_df_sheets[[str_sub(p_id, 1, 15)]] <- rhythm_df_sheets[[p_id]]
}

ERROR: Error in eval(expr, envir, enclos): object 'new_rhythm_df_sheets' not found


In [14]:
write_xlsx(rhythm_df_sheets, saving_file) # REMINDER: if you use personalized dominant period, change period_in_hour

Warning message in write_xlsx(rhythm_df_sheets, saving_file):
"Truncating sheet name(s) to 31 characters"

ERROR: Error: Error in writexl: failed to create workbook


In [19]:
saving_file <-  paste0(loc_save_file, dataset, '_', data_name,'.xlsx')

if (file.exists(saving_file))
{
  saved_rhythm_sheets_by_data = excel_sheets(path=saving_file)
  for (rhythm_sheet in saved_rhythm_sheets_by_data)
  {
      rhythm_df_sheets[[rhythm_sheet]] <- read_excel(path = saving_file, sheet = rhythm_sheet)
  }
}else
{
    saved_rhythm_sheets_by_data = c()
}

[1] "19329467-7c45-406b-b487-4dfd0"
[1] "1db1552c-5703-4400-a38e-8d3ba"
[1] "1f448103-5ef3-40e5-9131-a22cb"
[1] "21ab94d3-a40e-4c25-8410-ef07e"
[1] "240b79e1-6f67-4e8d-b8ca-3500c"
[1] "242dc7a7-083c-4b44-b65c-30e09"
[1] "28463428-d7ba-42c0-b40d-481fd"
[1] "2a62ecbe-e948-439f-a6a4-19374"
[1] "338d4215-9d6e-40b7-89bd-1467b"
[1] "3cbb9484-f795-4881-867e-724f0"
[1] "42355c53-f0ce-41f7-8d50-91ccd"
[1] "42a68d6a-3778-41cb-929b-6da08"
[1] "490461f8-3d56-4c88-8a4a-8ceb8"
[1] "4d2b899d-c221-4657-b74c-767e3"
[1] "4f91d234-b726-497b-aa81-df6b6"
[1] "50d1e47e-8b6c-47af-b5cb-e8742"
[1] "66c57f1a-de1f-415a-b79a-5701a"
[1] "6dce952d-f46c-408f-a8f1-ce636"
[1] "712a00af-4e8e-4236-b941-d6f9b"
[1] "7c78cdc7-915b-4883-9ca2-04c8b"
[1] "89209220-8ecf-4cd6-a9a4-aa713"
[1] "8a2e94a7-9dc3-4e67-a6d2-012da"
[1] "8dde8abc-014e-480a-9a07-c113c"
[1] "96d9d951-869c-4488-a649-c2ab6"
[1] "9885f7a8-e7d1-4b1c-8f85-3a4d0"
[1] "9cc29340-a44f-4d00-bac8-634cf"
[1] "a09fb0df-51ca-4fd9-9717-b7b36"
[1] "a34b4f63-6a7d-4a5f-bf11

In [13]:
library(readxl)

In [7]:
write_xlsx(rhythm_df_sheets, paste0(loc_save_file, dataset, '_', data_name,'.xlsx')) # REMINDER: if you use personalized dominant period, change period_in_hour

Warning message in write_xlsx(rhythm_df_sheets, paste0(loc_save_file, dataset, "_", :
"Truncating sheet name(s) to 31 characters"

In [63]:
newdata <- subset(df, subset=c((Timestamp > start_date) & (Timestamp < end_date)))
print(newdata)

                Timestamp HeartRatePPG
   1: 2020-02-04 21:35:49           80
   2: 2020-02-04 21:35:54           79
   3: 2020-02-04 21:35:59           77
   4: 2020-02-04 21:36:04           76
   5: 2020-02-04 21:36:09           75
  ---                                 
8812: 2020-02-06 21:35:09           79
8813: 2020-02-06 21:35:14           80
8814: 2020-02-06 21:35:19           81
8815: 2020-02-06 21:35:24           83
8816: 2020-02-06 21:35:29           82
